# Analyzing TSA Throughput Data and Historical Weather Information
- A Data Warehousing for Analytics undergraduate student group project for CIS 4400 course.

**Requirements**: https://docs.google.com/document/d/1djD0VMOxct1eiHj7tiv0HYmfNdgD4YXA1nGX4urJj_U/edit?usp=sharing

**Link to TSA Data Set**: https://www.tsa.gov/foia/readingroom?page=1

**Link to Weather API Data Set**: https://open-meteo.com/en/docs/historical-weather-api

**Data Dictionary**: https://docs.google.com/spreadsheets/d/1IMv8EoMX21I7BfsayR2Qs0bNYVQoMm-qADgmQC2n_Bk/edit?usp=sharing

# BigQuery Data Warehousing Script

**Link To Data**: https://data.cityofnewyork.us/City-Government/NYC-Citywide-Annualized-Calendar-Sales-Update/w2pb-icbu

**API end-point**: https://data.cityofnewyork.us/resource/w2pb-icbu.json

**Data Dictionary**: https://data.cityofnewyork.us/api/views/w2pb-icbu/files/8ed811b4-8238-4b5e-9acc-1e33d8705498?download=true&filename=Annualized_Calendar_Sales_Update%20Data_Dictionary.xlsx

**Cleaned Data Dictionary**: https://docs.google.com/spreadsheets/d/17XyGmnw2fZuTMCWVKB1XiWGHQuwqWOidm0w80lbIyjE/edit?usp=sharing



# Grant Required Permissions to your Google Cloud Service Account to Create a BigQuery Data Set

**1. Grant Permissions:**
- Go to the Google Cloud Console.
- Navigate to the IAM & Admin > IAM page.
- Locate the user account associated with the credentials you are using.
- Click "ADD IAM CONDITION"
- Under the "Role" field, select the "BigQuery Data Editor"
- Click "SAVE" to grant permissions.

**Note: The BigQuery Data Editor role allows your service account access to edit all the contents of datasets. This step is important for loading your dataset from Google Cloud to the BigQuery Data Warehouse.**

# Install the google-cloud-storage library

In [25]:
%%capture
pip install google-cloud-storage

In [5]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Install the google-cloud-bigquery library

In [24]:
%%capture
pip install google-cloud-bigquery

# Install upgraded bigquery library

In [23]:
%%capture
pip install --upgrade google-cloud-bigquery

# Install pandas gcsfs

In [22]:
%%capture
pip install pandas gcsfs

# Install pyarrow library

**NOTE: Once pyarrow is installed, you should be able to use the load_table_from_dataframe function without encountering the ValueError from the "Load Data into BigQuery Tables" Cell.**

**After installing pyarrow, you might need to restart your Python environment or Jupyter Notebook kernel before running the script again to ensure that the changes take effect.**

In [26]:
%%capture
pip install pyarrow

# Import the Python 'os' module

In [6]:
# CREATE A GOOGLE ACCESSKEY
import os
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/content/drive/MyDrive/Omakasi/GOOGLE_CLOUD_ACCESSKEY.json'

# Install pyarrow library

**NOTE: Once pyarrow is installed, you should be able to use the load_table_from_dataframe function without encountering the ValueError from the "Load Data into BigQuery Tables" Cell.**

**After installing pyarrow, you might need to restart your Python environment or Jupyter Notebook kernel before running the script again to ensure that the changes take effect.**

In [ ]:
pip install pyarrow

# Import the Python 'os' module

In [7]:
from google.cloud import bigquery

# Create BigQuery Dataset

In [8]:
from google.cloud import bigquery

# Creating a function for creating a BigQuery dataset with your file stored in your Google Cloud
def create_bigquery_dataset(project_id, dataset_name):
    bigquery_client = bigquery.Client(project=project_id)
    dataset_id = f"{project_id}.{dataset_name}"
    dataset = bigquery.Dataset(dataset_id)
    dataset.location = "US"
    bigquery_client.create_dataset(dataset)
    print(f"Dataset {dataset_id} created.")


project_id = 'your_project_id'
dataset_name = 'your_dataset_name'
create_bigquery_dataset(project_id, dataset_name)

Conflict: ignored

# Create Tables in BigQuery

In [14]:
from google.cloud import bigquery
from google.oauth2 import service_account

# Getting the path to the service account key file from the environment variable
service_account_path = os.environ.get('GOOGLE_APPLICATION_CREDENTIALS')

# Setting your Google Cloud credentials using the environment variable
credentials = service_account.Credentials.from_service_account_file(service_account_path)
# Initializing a BigQuery client
client = bigquery.Client(credentials=credentials, project=credentials.project_id)

# Defining your dataset and table names
dataset_name = 'jsoncme'
fact_table_name = 'travel_fact'
date_dim_table_name = 'dim_date'
weather_dim_table_name = 'dim_weather'
airport_dim_table_name = 'dim_airport'

# Creating the dataset
dataset_ref = client.dataset(dataset_name)
client.get_dataset(dataset_ref)

# Defining the schema for the fact table
fact_table_schema = [
    bigquery.SchemaField('travelid', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('date_id', 'INTEGER'),
    bigquery.SchemaField('airportid', 'INTEGER'),
    bigquery.SchemaField('weatherid', 'INTEGER'),
    bigquery.SchemaField('total', 'INTEGER'),
    bigquery.SchemaField('temperature_2m', 'FLOAT'),
    bigquery.SchemaField('precipitation', 'FLOAT'),
    bigquery.SchemaField('relative_humidity_2m', 'FLOAT'),
    bigquery.SchemaField('wind_speed_10m', 'FLOAT')
]

# Defining the schema for the date dimension table
date_dim_table_schema = [
    bigquery.SchemaField('date_id', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('datetime', 'DATE'),
    bigquery.SchemaField('day', 'INTEGER'),
    bigquery.SchemaField('month', 'INTEGER'),
    bigquery.SchemaField('year', 'INTEGER')
]
# Defining the schema for the weather table
weather_dim_table_schema = [
    bigquery.SchemaField('weatherid', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('city', 'STRING'),
    bigquery.SchemaField('weather_code', 'FLOAT')
]

# Defining the schema for the airport dimension table
dim_airport_schema = [
    bigquery.SchemaField('airportid', 'INTEGER', mode='REQUIRED'),
    bigquery.SchemaField('airportcode', 'STRING'),
    bigquery.SchemaField('state', 'STRING'),
]

# Creating the FACT_TABLE:
fact_table_ref = dataset_ref.table(fact_table_name)
try:
    client.get_table(fact_table_ref)
    print(f"Table {fact_table_name} already exists in the dataset {dataset_name}.")
except:
    fact_table = bigquery.Table(fact_table_ref, schema=fact_table_schema)
    client.create_table(fact_table)
    print(f"{fact_table_name} Created")

# Creating the DIM_DATE table:
date_dim_table_ref = dataset_ref.table(date_dim_table_name)
try:
    client.get_table(date_dim_table_ref)
    print(f"Table {date_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    date_dim_table = bigquery.Table(date_dim_table_ref, schema=date_dim_table_schema)
    client.create_table(date_dim_table)
    print(f"{date_dim_table_name} Created")

# Creating the DIM_WEATHER table:
weather_dim_table_ref = dataset_ref.table(weather_dim_table_name)
try:
    client.get_table(weather_dim_table_ref)
    print(f"Table {weather_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    weather_dim_table = bigquery.Table(weather_dim_table_ref, schema=weather_dim_table_schema)
    client.create_table(weather_dim_table)
    print(f"{weather_dim_table_name} Created")

# Creating the airport_dim_table:
airport_dim_table_ref = dataset_ref.table(airport_dim_table_name)
try:
    client.get_table(airport_dim_table_ref)
    print(f"Table {airport_dim_table_name} already exists in the dataset {dataset_name}.")
except:
    airport_dim_table = bigquery.Table(airport_dim_table_ref, schema=weather_dim_table_schema)
    client.create_table(airport_dim_table)
    print(f"{airport_dim_table_name} Created")

travel_fact Created
dim_date Created
dim_weather Created
dim_airport Created


# Read a dataset from your Google Cloud Storage into a Pandas DataFrame

In [16]:
import pandas as pd
from gcsfs import GCSFileSystem

# Replace with you actual bucket and file path
gcs_bucket = 'jsoncme'
gcs_file_path = 'Final_Merged.csv'

# Using Pandas to read the dataset from GCS into a DataFrame
df = pd.read_csv(f'gcs://{gcs_bucket}/{gcs_file_path}')

# Displaying the first few rows of the DataFrame
df.head()


,city,datetime,airportcode,airportname,state,checkpoint,total,temperature_2m,relative_humidity_2m,precipitation,weather_code,wind_speed_10m,date_id,day,month,year,weatherid,airportid,travelid,date
0,Anchorage,2023-05-28 00:00:00,ANC,Ted Stevens Anchorage International,AK,South Checkpoint,113,9.1645,70.430420,0.2,51.0,13.556282,20230528,28,5,2023,1,1,1,2023-05-28
1,Atlanta,2023-05-28 00:00:00,ATL,Hartsfield Atlanta International,GA,Main Checkpoint,142,18.4115,50.383205,0.0,1.0,20.898611,20230528,28,5,2023,2,2,2,2023-05-28
2,Aguadilla,2023-05-28 00:00:00,BQN,Rafael Hernandez,PR,Rafael Hernandez Air,268,25.0070,95.328810,0.0,1.0,3.219938,20230528,28,5,2023,3,3,3,2023-05-28
3,South Burlington,2023-05-28 00:00:00,BTV,Burlington International,VT,Main Checkpoint,0,21.1440,54.437480,0.0,0.0,6.989935,20230528,28,5,2023,4,4,4,2023-05-28
4,Arlington,2023-05-28 00:00:00,DCA,Washington Reagan National,VA,Concourse A,1,27.8830,43.359516,0.0,1.0,8.161764,20230528,28,5,2023,5,5,5,2023-05-28


# Load Data into BigQuery Tables

In [20]:
# Creating a function that uploads your data to BigQuery from a DataFrame
def upload_data_from_dataframe(df, table_ref):
    job_config = bigquery.LoadJobConfig()
    job_config.write_disposition = bigquery.WriteDisposition.WRITE_TRUNCATE
    job_config.autodetect = True
    job = client.load_table_from_dataframe(df, table_ref, job_config=job_config)
    job.result()

# Splitting your DataFrame into the respective dimension and fact DataFrames
# fact_df, date_dim_df, airport_dim_df, weather_dim_df = split_your_dataframe(df_transformed)
def split_df(df):
    fact_cols = [
    "travelid", "date_id", "airportid",
    "weatherid", "total",
    "temperature_2m", "precipitation",
    "relative_humidity_2m", "wind_speed_10m"]

    date_cols = [
    "date_id", "datetime", "day", "month", "year"]


    weather_cols = [
    "weatherid", "city", "weather_code"]

    airport_cols = [
    "airportid", "airportcode", "state"]

    fact_df = df[fact_cols]
    date_dim_df = df[date_cols]
    weather_dim_df = df[weather_cols]
    airport_dim_df = df[airport_cols]

    # Returning the split DataFrames
    return fact_df, date_dim_df, weather_dim_df, airport_dim_df

fact_df, date_dim_df, weather_dim_df, airport_dim_df = split_df(df)

# Uploading the data to BigQuery
upload_data_from_dataframe(fact_df, fact_table_ref)
upload_data_from_dataframe(date_dim_df, date_dim_table_ref)
upload_data_from_dataframe(weather_dim_df, weather_dim_table_ref)
upload_data_from_dataframe(airport_dim_df, airport_dim_table_ref)